In [16]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
# from scipy.stats import sharpe_ratio
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn
import math

In [17]:
# Define dataset
class TimeSeriesDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return torch.tensor(self.X[idx], dtype=torch.float), torch.tensor(self.y[idx], dtype=torch.float)

# # Define Transformer model
# class TransformerModel(nn.Module):
#     def __init__(self, input_dim, num_heads, num_layers, hidden_dim, embed_dim):
#         super(TransformerModel, self).__init__()
#         self.input_linear = nn.Linear(input_dim, embed_dim)
#         self.encoder_layer = nn.TransformerEncoderLayer(d_model=embed_dim, nhead=num_heads, dim_feedforward=hidden_dim)
#         self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)
#         self.linear = nn.Linear(input_dim, 1)

#     def forward(self, src):
#         output = self.transformer_encoder(src)
#         output = self.linear(output[:, -1, :])
#         return output
    
class TransformerModel(nn.Module):
    def __init__(self, input_dim, embed_dim, num_heads, num_layers):
        super().__init__()
        self.input_linear = nn.Linear(input_dim, embed_dim)
        encoder_layer = nn.TransformerEncoderLayer(d_model=embed_dim, nhead=num_heads)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.output_linear = nn.Linear(embed_dim, 1)

    def forward(self, x):
        x = self.input_linear(x)
        x = self.transformer_encoder(x)
        x = self.output_linear(x)
        return x.squeeze(-1)

In [18]:
# Load data
df = pd.read_csv('new.csv')
X = df.iloc[:, 1:].values
y = df.iloc[:, 0].values

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Hyperparameters
input_dim = X_train.shape[1]
embed_dim = 64  # Ensure this is divisible by num_heads
num_heads = 4
num_layers = 2
learning_rate = 0.001
batch_size = 32
epochs = 10

model = TransformerModel(input_dim, embed_dim, num_heads, num_layers)

# Initialize model, loss, and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training loop with time series cross-validation
train_size = int(0.5 * len(X_train))
val_size = 80
while train_size + val_size <= len(X_train):
    train_dataset = TimeSeriesDataset(X_train[:train_size], y_train[:train_size])
    val_dataset = TimeSeriesDataset(X_train[train_size:train_size+val_size], y_train[train_size:train_size+val_size])
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    for epoch in range(epochs):
        model.train()
        for inputs, targets in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs.unsqueeze(0)).squeeze()
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

        model.eval()
        with torch.no_grad():
            val_preds = []
            val_targets = []
            for inputs, targets in val_loader:
                outputs = model(inputs.unsqueeze(0)).squeeze()
                val_preds.extend(outputs.numpy())
                val_targets.extend(targets.numpy())

        r2 = r2_score(val_targets, val_preds)
        mse = mean_squared_error(val_targets, val_preds)
        accuracy = np.mean(np.sign(val_preds) == np.sign(val_targets))
        sharp_ratio = np.mean(np.abs(val_preds)) / np.std(val_targets)

        print(f'Epoch {epoch+1}/{epochs} - Train Size: {train_size} - R2: {r2:.4f} - MSE: {mse:.4f} - Accuracy: {accuracy:.4f} - Sharpe Ratio: {sharp_ratio}')

    train_size += val_size

# Test the model
test_dataset = TimeSeriesDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
model.eval()
with torch.no_grad():
    test_preds = []
    test_targets = []
    for inputs, targets in test_loader:
        outputs = model(inputs.unsqueeze(0)).squeeze()
        test_preds.extend(outputs.numpy())
        test_targets.extend(targets.numpy())

r2 = r2_score(test_targets, test_preds)
mse = mean_squared_error(test_targets, test_preds)
accuracy = np.mean(np.sign(test_preds) == np.sign(test_targets))
# try:
#     sharp_ratio = sharpe_ratio(test_preds - test_targets)
# except:
#     sharp_ratio = 'undefined'

# print(f'Test Results - R2: {r2:.4f} - MSE: {mse:.4f} - Accuracy:


Epoch 1/10 - Train Size: 796 - R2: -93.4021 - MSE: 0.0047 - Accuracy: 0.5625 - Sharpe Ratio: 9.8560209274292
Epoch 2/10 - Train Size: 796 - R2: -19.1252 - MSE: 0.0010 - Accuracy: 0.4250 - Sharpe Ratio: 4.1818389892578125
Epoch 3/10 - Train Size: 796 - R2: -30.5048 - MSE: 0.0016 - Accuracy: 0.4250 - Sharpe Ratio: 5.331697463989258
Epoch 4/10 - Train Size: 796 - R2: -1.9777 - MSE: 0.0001 - Accuracy: 0.4250 - Sharpe Ratio: 1.214908242225647
Epoch 5/10 - Train Size: 796 - R2: -3.0784 - MSE: 0.0002 - Accuracy: 0.5625 - Sharpe Ratio: 1.946038842201233
Epoch 6/10 - Train Size: 796 - R2: -14.1024 - MSE: 0.0007 - Accuracy: 0.4250 - Sharpe Ratio: 3.5637850761413574
Epoch 7/10 - Train Size: 796 - R2: -0.0299 - MSE: 0.0001 - Accuracy: 0.5625 - Sharpe Ratio: 0.3613438308238983
Epoch 8/10 - Train Size: 796 - R2: -6.0520 - MSE: 0.0003 - Accuracy: 0.4250 - Sharpe Ratio: 2.268578052520752
Epoch 9/10 - Train Size: 796 - R2: -1.3056 - MSE: 0.0001 - Accuracy: 0.4250 - Sharpe Ratio: 0.9508493542671204
Epoc

In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score, mean_squared_error
# from scipy.stats import sharpe_ratio

# Load data
df = pd.read_csv('new.csv')
X = df.iloc[:, 1:].values  # Features
y = df.iloc[:, 0].values   # Target

# Split data
train_size = int(len(X) * 0.8)
X_train, X_val = X[:train_size], X[train_size:]
y_train, y_val = y[:train_size], y[train_size:]

# Standardize data
mean, std = X_train.mean(axis=0), X_train.std(axis=0)
X_train = (X_train - mean) / std
X_val = (X_val - mean) / std

# Convert to PyTorch tensors
X_train_tensor, y_train_tensor = torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.float32)
X_val_tensor, y_val_tensor = torch.tensor(X_val, dtype=torch.float32), torch.tensor(y_val, dtype=torch.float32)

# Define model
class TransformerModel(nn.Module):
    def __init__(self, input_dim, embed_dim, num_heads, num_layers):
        super().__init__()
        self.input_linear = nn.Linear(input_dim, embed_dim)
        encoder_layer = nn.TransformerEncoderLayer(d_model=embed_dim, nhead=num_heads)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.output_linear = nn.Linear(embed_dim, 1)

    def forward(self, x):
        x = self.input_linear(x)
        x = self.transformer_encoder(x)
        x = self.output_linear(x)
        return x.squeeze(-1)

# Adjust these parameters as needed to ensure embed_dim is divisible by num_heads
input_dim = X_train.shape[1]
embed_dim = 64  # Ensure this is divisible by num_heads
num_heads = 4
num_layers = 2

model = TransformerModel(input_dim, embed_dim, num_heads, num_layers)

# Training parameters
batch_size = 32
learning_rate = 0.001
num_epochs = 10
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
loss_fn = nn.MSELoss()

# DataLoaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

# Training loop
for epoch in range(num_epochs):
    model.train()
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        y_pred = model(X_batch)
        loss = loss_fn(y_pred, y_batch)
        loss.backward()
        optimizer.step()

    model.eval()
    with torch.no_grad():
        y_val_pred = model(X_val_tensor)
        val_loss = loss_fn(y_val_pred, y_val_tensor)

    print(f'Epoch {epoch + 1}, Validation Loss: {val_loss.item()}')

# Evaluation metrics
y_val_pred = y_val_pred.numpy()
r2 = r2_score(y_val, y_val_pred)
mse = mean_squared_error(y_val, y_val_pred)
accuracy = np.mean(np.sign(y_val) == np.sign(y_val_pred))
# try:
#     sharpe = sharpe_ratio(y_val_pred - y_val)
# except ZeroDivisionError:
#     sharpe = 0.0

# print(f'R2: {r2}, MSE: {mse}, Accuracy: {accuracy}, Sharpe Ratio: {sharpe}')


Epoch 1, Validation Loss: 0.0035988595336675644
Epoch 2, Validation Loss: 0.000619834812823683
Epoch 3, Validation Loss: 0.0004354062839411199
Epoch 4, Validation Loss: 0.00041052736924029887
Epoch 5, Validation Loss: 0.00035818127798847854
Epoch 6, Validation Loss: 0.0004348757502157241
Epoch 7, Validation Loss: 0.00024258042685687542
Epoch 8, Validation Loss: 0.0003775167861022055
Epoch 9, Validation Loss: 0.0003454034449532628
Epoch 10, Validation Loss: 0.000256297760643065
